In [8]:
import pandas as pd 

from sqlalchemy import create_engine
from sqlalchemy import text

In [5]:
df = pd.read_csv("Path to CSV/sales_data_sample.csv", encoding='latin1')

In [7]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [10]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [11]:
# Here we are pushing our entinre dataset to a temporary Table named Sales

In [15]:
data = df.to_sql(name='Sales',con=temp_db, if_exists='replace', index=False)

2024-01-08 21:35:16,743 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-01-08 21:35:16,744 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-08 21:35:16,745 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-01-08 21:35:16,746 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-08 21:35:16,747 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2024-01-08 21:35:16,747 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-08 21:35:16,748 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Sales")
2024-01-08 21:35:16,749 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-08 21:35:16,751 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2024-01-08 21:35:16,751 INFO sqlalchemy.engine.Engine [raw sql] ('Sales',)
2024-01-08 21:35:16,752 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Sales")
2024-01-

In [17]:
with temp_db.connect() as conn:
    result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))

2024-01-08 21:37:23,886 INFO sqlalchemy.engine.Engine Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1
2024-01-08 21:37:23,888 INFO sqlalchemy.engine.Engine [generated in 0.00235s] ()


In [18]:
result.all()

[(10407, 14082.8)]

# SETTING UP OPEN AI API KEY

In [22]:
import openai
import os 

In [67]:
openai.api_key = os.getenv("<Your Secret Key>")

In [62]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs CHATGPT that we want to work with SQL Tables
    """
    prompt = '''### sqlite SQL table, with its properties:
#
# Sales ({})
#
'''.format(",".join(str(x) for x in df.columns))

    return prompt


### Taking Natural language request from end user and combining prompts

#### Will create an Input function to grab the user request

In [63]:
def prompt_input():
    nlp_text = input("Enter Information you want to obtain: ")
    return nlp_text

In [33]:
nlp_text = prompt_input()

In [64]:
def combine_prompts(df,query_prompt):
    definition = create_table_definition_prompt(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSelect"
    return definition+query_init_string

In [65]:
combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n#\n# Sales (ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n#\n### A query to answer: Sum of sales per Postal Code\nSelect'

##### Calling the Da-Vinci model and Parsing the request

In [ ]:

response = openai.Completion.create(
    model="text-davinci-003",
    prompt=combine_prompts(df,nlp_text),
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["#",";"]
)

In [ ]:
response

In [ ]:
def handle_response(response):
    query = response["choices"][0]["text"]
    if query.startswith(" "):
        query = "select" + query 
    return query


In [ ]:
handle_response(response)

In [ ]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))